In [1]:
import torch
import torch.nn as nn
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [23]:

excel_file = pd.ExcelFile("data.xlsx")

data = pd.DataFrame()


for sheet_name in excel_file.sheet_names:

    sheet_data = excel_file.parse(sheet_name)
    
    data = pd.concat([data, sheet_data])



In [24]:
data

,AT,V,AP,RH,PE
0,14.96,41.76,1024.07,73.17,463.26
1,25.18,62.96,1020.04,59.08,444.37
2,5.11,39.40,1012.16,92.14,488.56
3,20.86,57.32,1010.24,76.64,446.48
4,10.82,37.50,1009.23,96.62,473.90
...,...,...,...,...,...
9563,15.12,48.92,1011.80,72.93,462.59
9564,33.41,77.95,1010.30,59.72,432.90
9565,15.99,43.34,1014.20,78.66,465.96
9566,17.65,59.87,1018.58,94.65,450.93


In [25]:
train, test = train_test_split(data, test_size=0.2, random_state=42)
train, val = train_test_split(train, test_size=0.25, random_state=42)

# normalize the input features and the output variable
scaler_x = StandardScaler()
scaler_y = StandardScaler()

train_x = scaler_x.fit_transform(train.iloc[:,:-1].values)
train_y = scaler_y.fit_transform(train.iloc[:,-1].values.reshape(-1,1))

val_x = scaler_x.transform(val.iloc[:,:-1].values)
val_y = scaler_y.transform(val.iloc[:,-1].values.reshape(-1,1))

test_x = scaler_x.transform(test.iloc[:,:-1].values)
test_y = scaler_y.transform(test.iloc[:,-1].values.reshape(-1,1))

In [26]:
print(train_x)

[[ 1.28458868  1.56703203 -0.26703263 -0.05892276]
 [-0.81269777 -0.99184572  1.41444384 -0.71268882]
 [ 1.36087797  1.35110955 -0.53943182 -0.99845449]
 ...
 [ 1.96985393  1.85126454 -0.57474282 -1.05327764]
 [ 0.07600561  0.26600091  0.5098095   0.31044821]
 [ 0.09072811  0.24558642  0.18696602  0.66679869]]


In [27]:
print(train_y)

[[-1.6221037 ]
 [ 1.02173861]
 [-1.28738836]
 ...
 [-1.36858458]
 [-0.09573164]
 [-0.11792917]]


In [28]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(4, 16)
        self.fc2 = nn.Linear(16, 32)
        self.fc3 = nn.Linear(32, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [29]:
net = Net()

# define the loss function and the optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(net.parameters(), lr=0.01)

In [30]:
for epoch in range(1000):
    optimizer.zero_grad()
    outputs = net(torch.Tensor(train_x))
    loss = criterion(outputs, torch.Tensor(train_y))
    loss.backward()
    optimizer.step()

    if epoch % 100 == 0:
        val_outputs = net(torch.Tensor(val_x))
        val_loss = criterion(val_outputs, torch.Tensor(val_y))
        print("Epoch: {}, Loss: {}, Validation Loss: {}".format(epoch, loss.item(), val_loss.item()))

Epoch: 0, Loss: 0.970426619052887, Validation Loss: 0.8352819085121155
Epoch: 100, Loss: 0.06016606464982033, Validation Loss: 0.059740252792835236
Epoch: 200, Loss: 0.05666033923625946, Validation Loss: 0.05620970577001572
Epoch: 300, Loss: 0.055517349392175674, Validation Loss: 0.054999519139528275
Epoch: 400, Loss: 0.054500989615917206, Validation Loss: 0.05410519614815712
Epoch: 500, Loss: 0.05385856702923775, Validation Loss: 0.05362418293952942
Epoch: 600, Loss: 0.05324777960777283, Validation Loss: 0.05312761664390564
Epoch: 700, Loss: 0.05250966176390648, Validation Loss: 0.05260808393359184
Epoch: 800, Loss: 0.05205231532454491, Validation Loss: 0.052301738411188126
Epoch: 900, Loss: 0.05156160145998001, Validation Loss: 0.05183578282594681


In [31]:
test_outputs = net(torch.Tensor(test_x))
print(test_outputs)

tensor([[-1.1972],
        [ 0.8850],
        [-0.8563],
        ...,
        [-1.4702],
        [ 0.3486],
        [ 1.6085]], grad_fn=<AddmmBackward0>)


In [32]:
test_loss = criterion(test_outputs, torch.Tensor(test_y))
print("Test Loss: {}".format(test_loss.item()))

Test Loss: 0.05049177259206772
